<div style="text-align:center; font-family: Consolas; color: #FFFFFF;font-size: 28px;">Improve Algorithm of Formula Extraction</div>

- <div style = "font-family: Consolas; color: #00FF0F;font-size: 23px;" >Problem1: Formula not Matching</div>

  - <div style = "font-family: Consolas; color: #00FF0F;font-size: 20px;">Use F(ormula) from Grobid to find Proposals; Metric-1</div>

    - <div style = "font-family: Consolas; color: #00FF0F;font-size: 17px;">F in Grobid not perfect with wrong recog</div>
    - <div style = "font-family: Consolas; color: #00FF0F;font-size: 17px;">Some Text Similarity Algorithm to match F pairs</div>
    - <div style = "font-family: Consolas; color: #00FF0F;font-size: 17px;">Pay attention to the unicode in F</div>
    - <div style = "font-family: Consolas; color: #00FF0F;font-size: 17px;">Pay attention to the unrevelance charcter and noise</div>


  - <div style = "font-family: Consolas; color: #00FF0F;font-size: 21px;">Use Proposals and Match Further; Metric-2</div>
  
    - <div style = "font-family: Consolas; color: #00FF0F;font-size: 17px;">Consider the context(Give a Standard)</div>
    - <div style = "font-family: Consolas; color: #00FF0F;font-size: 17px;">Use Some Text Similarity Algorithm to match within a scope</div>


In [1]:
import os
import random

# fetch data
tex_root_path = "/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex"
tex_folds_rand_sample = random.sample(os.listdir(tex_root_path), 1)

xmls = [os.path.join("/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/xmls", f"{i}.tei.xml") for i in tex_folds_rand_sample]
latex = [[os.path.join(f"/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/{i}", j) for j in os.listdir(f"/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/{i}")]for i in tex_folds_rand_sample]


In [8]:
tex_folds_rand_sample

['1902.02632']

In [32]:
import re
from grobid_parser import GParser
import xml.etree.ElementTree as ET
from collections import OrderedDict

equation_xml_list = []
equation_xml_full_list = []
clean_data = ""
pdf_parser = GParser(parse_imgs=False)
for xml_path in xmls:
    pdf_name = os.path.basename(xml_path).replace(".tei.xml", ".pdf")
    pdf_parser.pdf_name = pdf_name
    tree = ET.parse(xml_path)
    root = tree.getroot()
    result = pdf_parser.parse_all(root, pdf_name, f"{pdf_name.strip('.pdf')}.json")
    body = result['body']
    for el in body:
        if el['el_type'] != "formula":
            if el['el_type'] == 's':
                clean_data = clean_data + el['txt'] + " "
            if "tail" in el:
                clean_data = clean_data + el['txt'] + el['tail']
            else:
                clean_data = clean_data + el['txt']
        else:
            equation_xml_list.append(el['txt'])
            clean_data = clean_data + "\\begin{equation}" + el['txt'] + "\\end{equation}"
    
            # 提取公式
            reg1 = r"(?<!\\)(?:((?<!\$)\${1,2}(?!\$))|(\\begin\{equation\})|(\\\()|(\\\[))(?(1)(.*?)(?<!\\)(?<!\$)\1(?!\$)|(?:([\s\S]*?)(?<!\\)(?:(?(2)\\end\{equation\}|(?(3)\\\)|(?(4)\\\]))))))"
            pattern1 = re.compile(reg1, re.MULTILINE)
            matches1 = pattern1.finditer(clean_data)   
            equ_spans = [match.span() for match in matches1]
            # 提取句点
            reg2 = r"(?<!No)(?<!\.g)(?<!\.e)(?<!\{)\.(?!g\.)(?!e\.)(?!\d)(?=\s)"
            pattern2 = re.compile(reg2, re.MULTILINE)
            matches2 = pattern2.finditer(clean_data)
            period_spans = [match.span() for match in matches2]

sentence_span = []
for equ_span in equ_spans:
    min_ = equ_span[0]
    max_ = equ_span[1]
    for i in range(len(period_spans)):
        if period_spans[i][1] <= min_:
            min = period_spans[i][1]
        if period_spans[i][0] >= max_:
            max = period_spans[i][0]
            break
    sentence_span.append((min, max+1))
sentence_span = list(OrderedDict.fromkeys(sentence_span).keys())
equation_xml_full_list = [clean_data[span[0]:span[1]] for span in sentence_span]

    # pre_txt = ""
    # post_txt = ""
    # for index, el in enumerate(body):
    #     if el['el_type'] == "formula":
    #         i_forward = 1
    #         i_backward = 1
    #         # 前向寻找
    #         while True:
    #             if "tail" in body[index - i_forward]:
    #                 pre_txt = body[index - i_forward]['txt'] + body[index - i_forward]['tail'] + pre_txt
    #             else:
    #                 pre_txt = body[index - i_forward]['txt'] + pre_txt
    #             if body[index - i_forward]['el_type'] == "s" and len(body[index - i_forward]['txt']) > 0:
    #                 break
    #             else:
    #                 i_forward += 1
    #         # 后向寻找
    #         while True:
    #             if "tail" in body[index + i_backward]:
    #                 post_txt = post_txt + body[index + i_backward]['txt'] + body[index + i_backward]['tail']
    #             else:
    #                 post_txt = post_txt + body[index + i_backward]['txt']
    #             if body[index + i_backward]['el_type'] == "s" and len(body[index + i_backward]['txt']) > 0:
    #                 break
    #             else:
    #                 i_backward += 1
    #         equation_txt = el['txt']
    #         full_txt = pre_txt + equation_txt + post_txt
    #         equation_xml_list.append({"pre_txt":pre_txt, "equation_txt":equation_txt, "post_txt":post_txt, "full_txt":full_txt})
    #         # 恢复初始态
    #         pre_txt = ""
    #         post_txt = ""
    #         i_forward = 1
    #         i_backward = 1
# equation_xml_full_list = [i['full_txt'] for i in equation_xml_list]
# equation_xml_full_list = list(OrderedDict.fromkeys(equation_xml_full_list).keys())


In [33]:
for i in equation_xml_full_list:
    print(i)
    print("=======================================================")


To describe the model, let Ω ⊂ IR 2 , be an open and bounded set; let also 0 < T < ∞ be a given final time of interest and denote t the times between [0, T ] and Q = Ω × (0, T ), the space-time cylinder and Γ = ∂Ω × (0, T ), the space-time boundary.Then, the system of equations we are considering is the following:
(1.1)\begin{equation}                               ∂N ∂t = r N − µ N N − β 1 N A − α H γ H N H, in Q, ∂A ∂t = ξ A ∆A + r A A 1 − A k A − (µ A + A )A − β 3 N A, in Q, ∂H ∂t = ξ H ∆H + νA − τ H H − γ H N H, in Q, ∂A ∂η (•) = ∂H ∂η (•) = 0, on Γ, N (•, 0) = N 0 (•), A(•, 0) = A 0 (•), H(•, 0) = H 0 (•), in Ω.\end{equation}
In [5], Fassoni studied a simplified model given by a system of ODE's describing the growth of a tumor and its effect in the normal tissue, together with the tissue response to the tumor and the effect of chemotherapeutic treatments.The aim of the authors was to provide some insights on the description of cancer onset and treat

In [34]:
# 思路：
# 获得公式及位于的index
# 获得段落句点的index
# 看公式落在哪两个句点之间
# 提取出来
import regex as re
from collections import OrderedDict
"""
# 找出句点
(?<!No)(?<!\.g)(?<!\.e)(?<!\{)\.(?!g\.)(?!e\.)
# 提取公式
(?<!\\)(?:((?<!\$)\${1,2}(?!\$))|(\\begin\{equation\})|(\\\()|(\\\[))(?(1)(.*?)(?<!\\)(?<!\$)\1(?!\$)|(?:([\s\S]*?)(?<!\\)(?:(?(2)\\end\{equation\}|(?(3)\\\)|(?(4)\\\]))))))
"""

for i in tex_folds_rand_sample:
    for j in os.listdir(os.path.join("/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/", i)):
        with open(os.path.join("/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/", i, j)) as fp:
            data = fp.read()
            # 去掉注释
            clean_data = re.sub(r'(?<!\\)%.*\n?', '', data)

            # 提取公式
            reg1 = r"(?<!\\)(?:((?<!\$)\${1,2}(?!\$))|(\\begin\{equation\})|(\\\()|(\\\[))(?(1)(.*?)(?<!\\)(?<!\$)\1(?!\$)|(?:([\s\S]*?)(?<!\\)(?:(?(2)\\end\{equation\}|(?(3)\\\)|(?(4)\\\]))))))"
            pattern1 = re.compile(reg1, re.MULTILINE)
            matches1 = pattern1.finditer(clean_data)   
            equ_spans = [match.span() for match in matches1]
            # 提取句点
            reg2 = r"(?<!No)(?<!\.g)(?<!\.e)(?<!\{)\.(?!g\.)(?!e\.)(?!\d)(?=\s)"
            pattern2 = re.compile(reg2, re.MULTILINE)
            matches2 = pattern2.finditer(clean_data)
            period_spans = [match.span() for match in matches2]


sentence_span = []
for equ_span in equ_spans:
    min_ = equ_span[0]
    max_ = equ_span[1]
    for i in range(len(period_spans)):
        if period_spans[i][1] <= min_:
            min = period_spans[i][1]
        if period_spans[i][0] >= max_:
            max = period_spans[i][0]
            break
    sentence_span.append((min, max+1))
sentence_span = list(OrderedDict.fromkeys(sentence_span).keys())
equation_latex_full_list = [clean_data[span[0]:span[1]] for span in sentence_span]

In [36]:
for i in equation_latex_full_list:
    print(i)
    print("=============================================================")



To describe the model,
let $\Omega \subset \R^{2}$, be an open and bounded set; let also  $0< T< \infty$ be a given final time  of interest and denote $t$ the times between $[0,T]$ and
$Q=\Omega\times(0,T)$, the space-time cylinder and
$\bar{\Gamma}=\partial \Omega\times(0,T)$,  the space-time boundary.
 Then, the system of equations we are considering is the following:
\begin{equation}
\label{0riginalEquations}
\left\{
\begin{array}{lcl}
\displaystyle
\frac{\partial N}{\partial t} = r_N - \mu_N N - \beta_1 N A - \alpha_H\gamma_H N H, & \textup{in}& Q,
\vspace{0.2cm}
\\
\displaystyle
\frac{\partial A}{\partial t} = \xi_A \Delta A + r_A A\left(1-\frac{A}{k_A}\right)-(\mu_A+\epsilon_A)A - \beta_3 NA , &\textup{in}& Q,
\vspace{0.2cm}
\\
\displaystyle
\frac{\partial H}{\partial t} = \xi_H \Delta H + \nu A - \tau_H H - \gamma_H NH , &\textup{in}& Q,
\vspace{0.2cm}
\\
\displaystyle \frac{\partial A}{\partial \eta} (\cdot) =\frac{\partial H}{\partial \eta} (\cdot) =0, &\textup{on}& \Gamma,


In [16]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint

for xml_eq in equation_xml_full_list:
    match_ = process.extractOne(xml_eq, equation_latex_full_list, scorer=fuzz.partial_ratio)
    latex_eq_ = match_[0]
    latex_score_ = match_[1]
    # 初步筛选
    if latex_score_ > 80 and len(latex_eq_) < 1.5*len(xml_eq):
        pprint(xml_eq)
        pprint("-----------------------------------------------------------------------------------------")
        pprint(latex_eq_)
        pprint(latex_score_)
        pprint("=======================================================================================")


('(1.2)\uf8f1 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f2 \uf8f4 \uf8f4 '
 '\uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f3 dN dt = r N − µ N N − β 1 N A − α N γ N '
 'DN, dA dt = r A A 1 − A k A − (µ A + A )A − β 3 N A − α A γ A DA, dD dt = µ '
 '− γ A DA − γ N DN − τ D,\n'
 'where N represents the normal cells in a given tissue of the human body, A '
 'represents the tumor cells in this tissue and D represents the concentration '
 'of a drug used to treat such a tumor.')
-----------------------------------------------------------------------------------------
('\n'
 '\\end{equation}\n'
 'where $N$ represents the normal cells in a given tissue of the human body, '
 '$A$ represents the tumor cells in this tissue and $D$ represents the '
 'concentration of a drug used to treat such a tumor.')
69
'======================================================================================='
('We will use standard notations for Sobolev spaces, i.e., given 1 ≤ p ≤ +∞ '
 'and k ∈ N, we denoteW k p (Ω) = 

In [24]:
fuzz.partial_ratio('(1.2)\uf8f1 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f2 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f4 \uf8f3 dN dt = r N − µ N N − β 1 N A − α N γ N DN, dA dt = r A A 1 − A k A − (µ A + A )A − β 3 N A − α A γ A DA, dD dt = µ − γ A DA − γ N DN − τ D,\nwhere N represents the normal cells in a given tissue of the human body, A represents the tumor cells in this tissue and D represents the concentration of a drug used to treat such a tumor.', ' The model studied in \\cite{Fassoni} is given by the following ODE system:\n\\begin{equation}\n\\label{início1}\n\\left\\{\n\\begin{array}{lcl}\n\\displaystyle\t\n\\frac{d N}{d t} = r_N - \\mu_N N - \\beta_1 N A - \\alpha_N\\gamma_N D N,\n\\vspace{0.2cm}\n\\\\\n\\displaystyle\t\n\\frac{d A}{d t} = r_A A\\left(1-\\frac{A}{k_A}\\right)-(\\mu_A+\\epsilon_A)A - \\beta_3 NA - \\alpha_A\\gamma_A D A,\n\\vspace{0.2cm}\n\\\\\n\\displaystyle\t\n\\frac{d D}{d t} = \\mu - \\gamma_A D A - \\gamma_N D N - \\tau D,\n\\end{array}\n\\right.\n\\end{equation}\nwhere $N$ represents the normal cells in a given tissue of the human body, $A$ represents the tumor cells in this tissue and $D$ represents the concentration of a drug used to treat such a tumor.')

63